# Perceptron de Rosenblatt

## Big Picture

%%html
<img src= 'https://storage.googleapis.com/open-courses/imagenes-5c33/DeepLearningTimeline.jpg' />

## Objetivo desde el punto de vista Matemático

%%html
<img src= 'https://storage.googleapis.com/open-courses/imagenes-5c33/DecisionBoundary.jpg' />

El el caso más simple, se parte de un conjunto de registros positivos y otro negativos en un espacio n-dimensional los que son separables por (infinitos) hiperplanos, encontrar uno de esos hiperplanos
Se tiene como input los puntos verdes y azules, generar como salida la recta roja

## Trabajo de Frank Rosenblatt

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/0925_rosenblatt_main.jpg' />

Frank Rosenblatt ’50, Ph.D. ’56, works on the “electronic profile analyzing computer” – a precursor to the perceptron.

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/0925_rosenblatt2.jpg' />

The first page of Rosenblatt's article, “The Design of an Intelligent Automaton,” in Research Trends, a Cornell Aeronautical Laboratory publication, Summer 1958.

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/1153px-Organization_of_a_biological_brain_and_a_perceptron.png' />

Organization of a biological brain and a perceptron.

## Neurona Biologica

<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/Neuron3.png' />

En el cerebro humano hay alrededor de 8.6 x 10^10  neuronas,
cada neurona en promedio se conecta con otras 7000, el cerebro humano tiene 1.6 x 10^13 parámetros


https://hms.harvard.edu/news/new-field-neuroscience-aims-map-connections-brain

https://www.beren.io/2022-08-06-The-scale-of-the-brain-vs-machine-learning/

## Concepto matemático de Perceptron

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/perceptron_elemental.jpg' />


%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/activation_function.png' />

Funcion de activacion escalón  {0,1}


## Algoritmo de Rosenblatt

### Intuición

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/geometric_intuition.png' />

Illustration of a Perceptron update. (Left:) The hyperplane defined by wt
 misclassifies one red (-1) and one blue (+1) point. (Middle:) The red point x
 is chosen and used for an update. Because its label is -1 we need to subtract x
 from wt
. (Right:) The udpated hyperplane wt+1=wt−x
 separates the two classes and the Perceptron algorithm has converged.


### Pseudocódigo del Algoritmo de Rosenblatt

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/PeceptronEntrenamiento.jpg' />

### Código en Python del Algoritmo de Rosenblatt

Código en Python que grafica como va evolucionando la recta que separa a los positivos de los negativos

In [ ]:
# conexion al Google Drive
from google.colab import drive
drive.mount('/content/.drive')
!mkdir -p "/content/.drive/My Drive/DMA"
!mkdir -p "/content/buckets"
!ln -s "/content/.drive/My Drive/DMA" /content/buckets/b1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import time

In [ ]:
# Ejemplo, usted deberá generar sus propios puntos en el plano
#  los que mandatoriamente deben ser separables por una recta
#  los puntos traen incorporada la dimension artificial x0
#  <x0, x1, x2>    donde    x0 = 1

registros = [ [1, 0.7, 1.3], [1, 2.0, 1.1], [1, 1.0, 1.9],
            [1, 3.0, 1.0], [1, 1.5, 2.1] ]
clases = [0,0,0,1,1]


In [ ]:
# Paso las listas a numpy
X = np.array(registros)
Y = np.array(clases)

In [ ]:
# Tamano datos
X_row = X.shape[0]
X_col = X.shape[1]

In [ ]:
class perceptron_plot:
    def __init__(self, X, Y, delay) -> None:
        self.X = X
        self.Y = Y
        self.delay = delay
        x1_min = np.min(X[:,1])
        x2_min = np.min(X[:,2])
        x1_max = np.max(X[:,1])
        x2_max = np.max(X[:,2])
        self.x1_min = x1_min - 0.75*(x1_max - x1_min)
        self.x1_max = x1_max + 0.75*(x1_max - x1_min)
        self.x2_min = x2_min - 0.75*(x2_max - x2_min)
        self.x2_max = x2_max + 0.75*(x2_max - x2_min)
        self.fig = plt.figure(figsize = (10,8))
        self.ax = self.fig.subplots()
        self.ax.set_xlim(self.x1_min, self.x1_max, auto=False)
        self.ax.set_ylim(self.x2_min, self.x2_max, auto=False)

    def graficar(self, W, epoch, fila) -> None:
        display.clear_output(wait =True)
        plt.cla()
        #self.ax = self.fig.subplots()

        self.ax.set_xlim(self.x1_min, self.x1_max)
        self.ax.set_ylim(self.x2_min, self.x2_max)
        plt.title( 'epoch ' + str(epoch) + '  reg ' + str(fila) )
        # ploteo puntos positivos
        self.ax.plot(self.X[self.Y==1,1], self.X[self.Y==1,2], 'o', color="green", markersize=10)
        # ploteo puntos negativos
        self.ax.plot(self.X[self.Y==0,1], self.X[self.Y==0,2], 'o', color="blue", markersize=10)

        # Sobreploteo el punto que no coincidio
        if(fila>=0):
            self.ax.plot(self.X[fila,1], self.X[fila,2], 'o',
                         color= ('green' if self.Y[fila]==1 else 'blue'),
                         markersize= 12, markeredgecolor= 'red')

        #dibujo le recta
        vx2_min = -(W[1]*self.x1_min + W[0])/W[2]
        vx2_max = -(W[1]*self.x1_max + W[0])/W[2]

        self.ax.plot([self.x1_min, self.x1_max],
                     [vx2_min, vx2_max],
                     linewidth = 2,
                     color = 'red',
                     alpha = 0.5)

        display.display(plt.gcf())
        #plt.cla()
        time.sleep(self.delay)


    def graficarVarias(self, W, epoch, fila) -> None:
        display.clear_output(wait =True)
        plt.cla()
        #self.ax = self.fig.subplots()

        self.ax.set_xlim(self.x1_min, self.x1_max)
        self.ax.set_ylim(self.x2_min, self.x2_max)
        plt.title( 'epoch ' + str(epoch) + '  reg ' + str(fila))
        # ploteo puntos positivos
        self.ax.plot(self.X[self.Y==1,1], self.X[self.Y==1,2], 'o', color="green", markersize=10)
        # ploteo puntos negativos
        self.ax.plot(self.X[self.Y==-1,1], self.X[self.Y==-1,2], 'o', color="blue", markersize=10)
        self.ax.plot(self.X[self.Y==0,1], self.X[self.Y==0,2], 'o', color="blue", markersize=10)

        # Sobreploteo el punto que no coincidio
        if(fila>=0):
            self.ax.plot(self.X[fila,1], self.X[fila,2], 'o',
                         color= ('green' if self.Y[fila]==1 else 'blue'),
                         markersize= 12, markeredgecolor= 'red')

        # dibujo las rectas
        for i in range(len(x0)):
            #vx2_min = -(W[0,i]*self.x1_min + x0[i])/W[1,i]
            #vx2_max = -(W[0,i]*self.x1_max + x0[i])/W[1,i]
            vx2_min = -(W[i,1]*self.x1_min + W[i,0])/W[i,2]
            vx2_max = -(W[i,1]*self.x1_max + W[i,0])/W[i,2]

            self.ax.plot([self.x1_min, self.x1_max],
                         [vx2_min, vx2_max],
                         linewidth = 2,
                         color = 'red',
                         alpha = 0.5)

        display.display(plt.gcf())
        #plt.cla()
        time.sleep(self.delay)


In [ ]:
# Incializo la recta azarosamente

# seteo de la semilla aleatoria
np.random.seed(102191) # mi querida random seed para que las corridas sean reproducibles

In [ ]:
# inicializo < x0, W >
W = np.array( np.random.uniform(-0.5, 0.5, size=X_col))

In [ ]:
# Limite de lo que estoy dispuesto a trabajar
epoch_limit = 500    # para terminar si no converge

In [ ]:
# controla la velocidad de convergencia
learning_rate = 0.01

In [ ]:
# controla cada cuanto se grafica un epoch
demora_impresion = 1.0  # segundos

In [ ]:
# inicializaciones del buble
modificados = 1      # lo debo poner algo distinto a 0 la primera vez
epoch = 0

In [ ]:
# el bucle principal del algoritmo de Rosenblatt

grafico = perceptron_plot(X=X, Y=Y, delay=demora_impresion)

# bucle principal del algoritmo de Rosenblatt
# continuo mientras en la iteración anterior modifique algo  y NO llegué al límite de epochs
while (modificados and (epoch < epoch_limit)):
    epoch += 1
    modificados = 0  # lo seteo en cero

    # recorro siempre TODOS los registros de entrada
    for fila in range(X_row):
        # fila es el registro actual
        x = X[fila,]
        clase = Y[fila]
        # calculo el estimulo suma, producto interno
        estimulo = W @ x

        # funcion de activacion, a lo bruto con un if
        if(estimulo>0):
            y = 1
        else:
            y = 0

        # solo si la prediccion es incorrecta  actualizo  < x0, W >
        gradiente = clase -y
        if(gradiente != 0 ):
            modificados += 1  # encontre un registro que esta mal clasificado
            # actualizo W y x0
            W = W + learning_rate * gradiente * x
            grafico.graficar(W, epoch, fila) # actualizo grafico


In [ ]:
grafico = perceptron_plot(X, Y, 0)
grafico.graficar(W, epoch, -1)

In [ ]:
# la cantidad de epochs necesarias hasta encontrar una solucion
print("Para converger hicieron falta epochs=",epoch)

In [ ]:
# el vector solución W, que incluye a  X0
print( "La solucion es W = ", W)

In [ ]:
# la norma (longitud) del vector W
print( ' ║W║  = ' , np.linalg.norm(W) )